In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

'''import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"import os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [2]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

# Define paths
REAL_FRAMES_DIR = '/kaggle/input/dfdc-dataset/Dataset/real'  # Directory containing frames for real videos
FAKE_FRAMES_DIR = '/kaggle/input/dfdc-dataset/Dataset/fake'  # Directory containing frames for fake videos
OUTPUT_FEATURES_PATH = '/kaggle/working/'  # Path where you want to save the features

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

def preprocess(image):
    # Resize and normalize
    image_resized = cv2.resize(image, (224, 224))  # Resize to model input size
    image_resized = image_resized.astype(np.float32) / 255.0  # Normalize to [0,1]
    image_resized = (image_resized - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]  # Standardize (ImageNet stats)
    image_resized = np.transpose(image_resized, (2, 0, 1))  # Change to (C, H, W)
    image_tensor = torch.from_numpy(image_resized).float()  # Convert to tensor
    return image_tensor

# Load and display frames (for debugging)
def load_and_display_frames(frames_dir, num_frames=1):
    frame_files = sorted([os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith('.png')])

    if len(frame_files) == 0:
        print(f"Warning: No frames found in {frames_dir}")
        return None
    
    # Display the first frame for visual check
    frame = cv2.imread(frame_files[0])
    if frame is None:
        print(f"Warning: Failed to load frame {frame_files[0]}")
        return None
    
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title(f"Sample Frame from {frames_dir}")
    plt.show()

    return frame_files

# Test feature extraction on a single frame
def test_feature_extraction(frames_dir):
    frame_files = sorted([os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith('.png')])

    if len(frame_files) == 0:
        print(f"Warning: No frames found in {frames_dir}")
        return None
    
    # Load the first frame
    frame = cv2.imread(frame_files[0])
    if frame is None:
        print(f"Warning: Failed to load frame {frame_files[0]}")
        return None
    
    processed_frame = preprocess(frame)
    processed_frame = processed_frame.unsqueeze(0)  # Add batch dimension

    # Extract features
    with torch.no_grad():
        features = model(processed_frame)
        features = features.squeeze().numpy()

    print(f"Feature shape: {features.shape}")
    return features

# Extract features from all frames in a directory
def extract_features_from_directory(frames_dir, label):
    features_list = []
    labels_list = []

    frame_files = sorted([os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith('.png')])

    for i, frame_file in enumerate(frame_files):
        # Load and preprocess frame
        frame = cv2.imread(frame_file)
        if frame is None:
            print(f"Warning: Failed to load frame {frame_file}")
            continue

        processed_frame = preprocess(frame)
        processed_frame = processed_frame.unsqueeze(0)  # Add batch dimension

        # Extract features
        with torch.no_grad():
            features = model(processed_frame)
            features = features.squeeze().numpy()

        features_list.append(features)
        labels_list.append(label)

        if i % 100 == 0:
            print(f"{i} frames processed from {frames_dir}")

    return np.array(features_list), np.array(labels_list)

# Save features to numpy files
def save_features_and_labels(real_dir, fake_dir, output_path):
    # Extract features for real frames (label = 0)
    print("Processing real frames...")
    real_features, real_labels = extract_features_from_directory(real_dir, label=0)

    # Extract features for fake frames (label = 1)
    print("Processing fake frames...")
    fake_features, fake_labels = extract_features_from_directory(fake_dir, label=1)

    # Concatenate features and labels
    features = np.concatenate([real_features, fake_features], axis=0)
    labels = np.concatenate([real_labels, fake_labels], axis=0)

    print(f"Features array shape: {features.shape}")
    print(f"Labels array shape: {labels.shape}")

    # Save to disk
    np.save(os.path.join(output_path, 'features.npy'), features)
    np.save(os.path.join(output_path, 'labels.npy'), labels)

    print(f"Features and labels saved to {output_path}")

# Example usage
save_features_and_labels(REAL_FRAMES_DIR, FAKE_FRAMES_DIR, OUTPUT_FEATURES_PATH)


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 168MB/s]


Processing real frames...
0 frames processed from /kaggle/input/dfdc-dataset/Dataset/real
100 frames processed from /kaggle/input/dfdc-dataset/Dataset/real
200 frames processed from /kaggle/input/dfdc-dataset/Dataset/real
300 frames processed from /kaggle/input/dfdc-dataset/Dataset/real
400 frames processed from /kaggle/input/dfdc-dataset/Dataset/real
500 frames processed from /kaggle/input/dfdc-dataset/Dataset/real
600 frames processed from /kaggle/input/dfdc-dataset/Dataset/real
700 frames processed from /kaggle/input/dfdc-dataset/Dataset/real
Processing fake frames...
0 frames processed from /kaggle/input/dfdc-dataset/Dataset/fake
100 frames processed from /kaggle/input/dfdc-dataset/Dataset/fake
200 frames processed from /kaggle/input/dfdc-dataset/Dataset/fake
300 frames processed from /kaggle/input/dfdc-dataset/Dataset/fake
400 frames processed from /kaggle/input/dfdc-dataset/Dataset/fake
500 frames processed from /kaggle/input/dfdc-dataset/Dataset/fake
600 frames processed from /k